In [ ]:
import requests
import json
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

In [ ]:
MAX_PAIRS = 25
CAPITAL= 100

In [ ]:
def get_coinmarketcap_top100():
    # Get API key from environment variables
    cmc_key = os.getenv('CMC_KEY')
    if not cmc_key:
        raise ValueError("CoinMarketCap API key not found in environment variables")

    # Specify the endpoint
    endpoint = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'

    # Specify the parameters
    parameters = {
        'sort': 'market_cap',
        'sort_dir': 'desc',
        'cryptocurrency_type': 'all',
        'limit': 100
    }
    # Send the request
    headers = {'X-CMC_PRO_API_KEY': cmc_key}
    response = requests.get(endpoint, headers=headers, params=parameters)

    # Parse the response
    data = json.loads(response.text)
    coins = data['data']
    # Filter out stablecoins and create pairs
    stablecoins = ['USDC', 'USDT', 'BUSD', 'DAI', 'TUSD', 'FDUSD']
    top100_symbols = [coin['symbol'] + "USDC" for coin in data['data'] 
                     if coin['symbol'] not in stablecoins]
    # print(f'Top 100 Symbols: {top100_symbols}\n')
    return top100_symbols

In [ ]:
def get_binance_usdt_pairs():
    url = 'https://api.binance.com/api/v3/exchangeInfo'
    response = requests.get(url)
    data = response.json()
    stablecoins = ['USDC', 'USDT', 'BUSD', 'DAI', 'TUSD']
    usdt_pairs = [pair['symbol'] for pair in data['symbols'] 
                 if pair['quoteAsset'] == 'USDC' 
                 and pair['status'] == 'TRADING'
                 and pair['baseAsset'] not in stablecoins]
    # print(f'USDT Pairs: {usdt_pairs}\n')
    return usdt_pairs

In [ ]:
def find_common_pairs(top100_symbols, usdt_pairs):
    common_pairs = set(top100_symbols) & set(usdt_pairs)
    # Convert to list and take only the first MAX_PAIRS pairs
    # Since common_pairs is a set, we sort it to ensure consistent results
    return list(sorted(common_pairs))[:MAX_PAIRS]

In [ ]:
top100_symbols = get_coinmarketcap_top100()
usdt_pairs = get_binance_usdt_pairs()
common_pairs = find_common_pairs(top100_symbols, usdt_pairs)

print(f"Top {MAX_PAIRS} cryptocurrencies in the top 100 on CoinMarketCap which are tradeable on Binance (USDT pairs): \n {common_pairs}")
print(f"Number of pairs selected: {len(common_pairs)}")

In [ ]:
capital_per_pair = CAPITAL / len(common_pairs)
print(f"With ${CAPITAL} total capital, you can spend ${capital_per_pair:.2f} on each of the {len(common_pairs)} pairs")

In [ ]:
# TODO: use ccxt to get the usd-equivalent balance of my account
# TODO: use capital_per_pair to calculate the difference between the current balance and capital_per_pair
# TODO: execute buy or sell to make the balance equal to capital_per_pair
# TODO: buy any coins that are not in the account
# TODO: sell any coins that are not in the common_pairs list

In [ ]:
import ccxt

# Get API credentials from environment variables
binance_api_key = os.getenv('BINANCE_API_KEY')
binance_api_secret = os.getenv('BINANCE_API_SECRET')

if not binance_api_key or not binance_api_secret:
    raise ValueError("Binance API credentials not found in environment variables")

# Initialize Binance exchange
exchange = ccxt.binance({
    'apiKey': binance_api_key,
    'secret': binance_api_secret,
    'enableRateLimit': True
})

# Fetch balance
balance = exchange.fetch_balance()

# Calculate USD equivalent for each coin in common_pairs
usd_balances = {}
for pair in common_pairs:
    # Extract base currency (remove 'USDC' from the end)
    base_currency = pair[:-4]
    
    # Get the balance for this currency
    amount = float(balance['total'].get(base_currency, 0))
    
    if amount > 0:
        # Fetch current price in USDC
        try:
            ticker = exchange.fetch_ticker(pair)
            price = ticker['last']
            usd_value = amount * price
            usd_balances[base_currency] = {
                'amount': amount,
                'usd_value': usd_value
            }
        except:
            print(f"Could not fetch price for {pair}")

# Print results
for currency, data in usd_balances.items():
    print(f"{currency}: {data['amount']:.8f} coins = ${data['usd_value']:.2f}")

total_usd_value = sum(data['usd_value'] for data in usd_balances.values())
print(f"\nTotal portfolio value: ${total_usd_value:.2f}")